In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt

# Video operations function
def process_video(video):
    # Resize the resolution of the video become 600 x 500
    resize = cv2.resize(video, (600,500))

    # Convert the video BGR (Blue-Green-Red) color space to grayscale
    gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)

    # Convert the video color back to a 3-channel (RGB) image
    gray_3d = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Apply Gaussian filter to the video
    gaussian = cv2.GaussianBlur(resize, (31,31), 0)

    # Convert the video to the HSV color space using tensorflow
    vid = tf.convert_to_tensor(resize, dtype=tf.float32) / 250.0
    hsv = tf.image.rgb_to_hsv(vid)

    # Convert the scaled HSV tensor back to the uint8 data type or else it won't be visible
    hsv = tf.cast(hsv * 250, dtype=tf.uint8)
    hsv_numpy = hsv.numpy()

    # Convert the video to the HSV color space using OpenCV
    # hsv_numpy = cv2.cvtColor(resize, cv2.COLOR_BGR2HSV)


    # Creates a 5x5 kernel with all elements set to 1
    kernel = np.array([[1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1]])

    # Normalizes the kernel
    kernel = kernel/sum(kernel)

    # Apply low-pass filtering to the video
    low = cv2.filter2D(resize,-1,kernel)

    # Apply high-pass filtering to the video
    high = resize - cv2.GaussianBlur(resize, (21, 21), 3)+127

    # Apply histogram equalization
    equalized = cv2.equalizeHist(cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY))
    equalized_3d = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

    # Stack videos in a 3x2 grid
    row1 = np.hstack((resize, gaussian, hsv_numpy))
    row2 = np.hstack((low, high, equalized_3d))
    combined = np.vstack((row1, row2))
    return combined

# Load the video from file
clip = VideoFileClip("homework_1_test_video.mp4")

# Process the video
processed_clip = clip.fl_image(process_video)

# Save the video into mp4 format with name "Output1"
processed_clip.write_videofile("Output1.mp4")

Moviepy - Building video Output1.mp4.
MoviePy - Writing audio in Output1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Output1.mp4



Moviepy - Done !
Moviepy - video ready Output1.mp4


In [ ]:
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

In [ ]:
import os
import cv2
import tensorflow as tf
import urllib
import numpy as np
import yt_dlp
import matplotlib.pyplot as plt
from moviepy.editor import VideoClip, VideoFileClip, clips_array

# Import Youtube URL
youtube_url = "https://www.youtube.com/watch?v=PHqhEgkGfrs"
data = urllib.parse.urlparse(youtube_url)
query = urllib.parse.parse_qs(data.query)
ID = query["v"][0]
video = 'https://www.youtube.com/watch?v={}'.format(str(ID))

ydl_options = {
    'format': 'best',
}

with yt_dlp.YoutubeDL(ydl_options) as ydl:
    info = ydl.extract_info(video, download=False)
    url = info.get("url", None)

capture = cv2.VideoCapture(url)
mili = 1000
start = 0
end = 9
total = start + end

frames = []

while True and capture.get(cv2.CAP_PROP_POS_MSEC)<=total*mili:
    ret, image = capture.read()
    if not ret:
        break

    # Resize the resolution of the video become 600 x 500
    resize = cv2.resize(image, (600,500))

    # Creates a 5x5 kernel with all elements set to 1
    kernel = np.array([[1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1]])

    # Normalizes the kernel
    kernel = kernel/sum(kernel)

    # Apply low-pass filtering to the video
    low = cv2.filter2D(resize,-1,kernel)

    # Apply high-pass filtering to the video
    high = resize - cv2.GaussianBlur(resize, (21, 21), 3)+127

    # Apply histogram equalization
    equalized = cv2.equalizeHist(cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY))

    # Convert the video color back to a 3-channel (RGB) image
    equaliz = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

    # Stack videos in a 3x2 grid
    combined = np.hstack((low, high, equaliz))

    frames.append(combined)


def process_image(image):
    # Resize the resolution of the video become 600 x 500
    resize = cv2.resize(image, (600,500))

    # Convert the video BGR (Blue-Green-Red) color space to grayscale
    gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)

    # Convert the video color back to a 3-channel (RGB) image
    gray_3d = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Apply Gaussian filter to the video
    gaussian = cv2.GaussianBlur(resize, (31,31), 0)

    # Convert the video to the HSV color space using tensorflow
    vid = tf.convert_to_tensor(resize, dtype=tf.float32) / 250.0
    hsv = tf.image.rgb_to_hsv(vid)

    # Convert the scaled HSV tensor back to the uint8 data type or else it won't be visible
    hsv = tf.cast(hsv * 250, dtype=tf.uint8)
    hsv_numpy = hsv.numpy()

    combined = np.hstack((resize, gaussian, hsv_numpy))
    return combined

# Input video
video1 = VideoFileClip('homework_1_test_video.mp4').fl_image(process_image)
video2 = VideoClip(lambda t: frames[int(t*20)], duration=len(frames)/20.0)

# Make sure the videos have the same duration
duration = min(video1.duration, video2.duration)
clip1 = video1.subclip(0, duration)
clip2 = video2.subclip(0, duration)

# Merge the videos side by side
final_clip = clips_array([[clip1],[clip2]])

# Write the result to a file
final_clip.write_videofile('YoutubeOutput.mp4', fps = 30)

In [ ]:
!pip install pyttsx3
!pip install moviepy
!pip install gTTS
!pip install yt-dlp
!pip install moviepy

!apt install imagemagick

!apt install libmagick++-dev

!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml
!pip install espeak-py
!wget https://github.com/taveevut/Windows-10-Fonts-Default/raw/master/msjh.ttc

In [5]:
from gtts import gTTS
from PIL import ImageFont, ImageDraw, Image
from moviepy.editor import *
from moviepy.video.tools.segmenting import findObjects
from moviepy.video.tools.subtitles import SubtitlesClip
import pyttsx3
import moviepy.video.fx.all as vfx
import numpy as np
import cv2

#Subtitles
subtitles = '''
幾乎整個影片幾乎完全由人工智慧生成,
都是用python程式碼寫的,
視頻來源是在左上角視頻,
中間頂部的影片使用 GAUSSIAN BLUR 進行過濾,
右上角影片使用tensorflow轉換為HSV,
在左下角，影片使用 KERNEL 過濾為 Low-Pass Filtering,
在底部中間，影片被過濾為 High-Pass Filtering,
最後右下角的影片透過 Histogram Equalization 進行了增強.
Thank You for Watching!!
'''

source_video_filename     = "/content/YoutubeOutput.mp4"
background_music_filename = "/content/calm-background-for-video-121519.mp3"
target_video_with_subtitle= "VideoSubtitle.mp4"

lines = [msg for msg in subtitles.split('\n') if len(msg)>0]
speech= []

# Text To Speech Loop reading every line in the lines
for i,msg in enumerate(lines):
    gttsObj = gTTS(text = msg, lang = 'zh-TW', slow =False)
    gttsObj.save('/content/voice{:04d}.mp3'.format(i))
    speech.append(AudioFileClip('/content/voice{:04d}.mp3'.format(i)))

# Calculate the start and end time of each narration
duration       = np.array([0]+[s.duration for s in speech])
cumduration    = np.cumsum(duration)
total_duration = int(cumduration[-1])+4

# Generate narration subtitles, using msjh font
generator = lambda txt: TextClip(txt, font='/content/msjh.ttc', fontsize = 55, color = 'white')
subtitles = SubtitlesClip([((cumduration[i],cumduration[i+1]),s) for i,s in enumerate(lines)], generator)

# Adjust the video duration so that the video playback time is longer than the entire narration.
clip = VideoFileClip(source_video_filename)
clip = clip.fx(vfx.speedx,clip.duration/total_duration)

# Generate the final video with subtitles.
final_clip = CompositeVideoClip([clip, subtitles.set_pos(('center','bottom'))])

# Adding Background music, length is same as video and turn down the volume.
bgm = AudioFileClip(background_music_filename)
bgaudio = bgm.subclip(bgm.duration-total_duration).volumex(0.2)


# Combine the audio of the background music and the Narrative
voices = concatenate_audioclips(speech)
clip.audio = CompositeAudioClip([bgaudio,voices])
final_clip = final_clip.set_audio(clip.audio)

# Output
final_clip.write_videofile(target_video_with_subtitle)

Moviepy - Building video VideoSubtitle.mp4.
MoviePy - Writing audio in VideoSubtitleTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video VideoSubtitle.mp4



t: 100%|█████████▉| 1586/1590 [02:54<00:00,  9.69it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/YoutubeOutput.mp4, 5400000 bytes wanted but 0 bytes read,at frame 297/298, at time 9.90/9.93 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready VideoSubtitle.mp4
